This notebook returns offers for brands that fetch has in its database

Approach:

- If a brand has associated offers in the final DataFrame created by merging brands_df, category_df, and retail_df, we return all non-NaN offers for that brand. But, in cases where a brand has only NaN offers, we follow a two-step process:

- Create a dictionary where keys are brand names with non-NaN offers, and values are sets of products associated with those brands.

- Calculate Jaccard similarity between the set of products for the brand with all NaN offers and the sets of products for brands with non-NaN offers. We then order these brands based on their similarity scores and return the top offers from the most similar brands.

In [2]:
#Import libraries
import numpy as np
import pandas as pd

In [3]:
# Import dataset
brands_df = pd.read_csv('/content/drive/MyDrive/fetch/brand_category.csv')
category_df = pd.read_csv('/content/drive/MyDrive/fetch/categories.csv')
retail_df = pd.read_csv('/content/drive/MyDrive/fetch/offer_retailer.csv')

In [4]:
#brands_df
brands_df['BRAND'] = brands_df['BRAND'].str.lower()
brands_df['BRAND_BELONGS_TO_CATEGORY'] = brands_df['BRAND_BELONGS_TO_CATEGORY'].str.lower()

In [5]:
#categories_df
category_df['PRODUCT_CATEGORY'] = category_df['PRODUCT_CATEGORY'].str.lower()
category_df['IS_CHILD_CATEGORY_TO'] = category_df['IS_CHILD_CATEGORY_TO'].str.lower()

In [6]:
#retail_df
retail_df['RETAILER'] = retail_df['RETAILER'].str.lower()
retail_df['BRAND'] = retail_df['BRAND'].str.lower()

In [7]:
# Merge the dataframes based on the common column and keep only the required column
merged_df = brands_df.merge(category_df[['PRODUCT_CATEGORY', 'IS_CHILD_CATEGORY_TO']],
                            left_on='BRAND_BELONGS_TO_CATEGORY',
                            right_on='PRODUCT_CATEGORY',
                            how='left')

# Drop the columns that are not needed
merged_df.drop(columns=['PRODUCT_CATEGORY', 'RECEIPTS'], inplace=True)

# Rename the new column if needed
merged_df.rename(columns={'IS_CHILD_CATEGORY_TO': 'Product_category'}, inplace=True)
merged_df.rename(columns={'BRAND_BELONGS_TO_CATEGORY': 'Product'}, inplace=True)


# Save the merged dataframe to a new CSV file if needed
merged_df.to_csv('merged_brands.csv', index=False)

In [8]:
# Rename caseys gen store to caseys general store
merged_df['BRAND'] = merged_df['BRAND'].replace({'caseys gen store': 'caseys general store'})

In [9]:
# Merge the dataframes based on the common 'BRAND' column using a left join
merged_with_offers_df = merged_df.merge(retail_df, on='BRAND', how='left')

# Save the merged dataframe with offers to a new CSV file if needed
merged_with_offers_df.to_csv('merged_with_offers.csv', index=False)

In [10]:
# Reorder columns of merged_with_offers_df
ordered_df = merged_with_offers_df[['RETAILER', 'BRAND', 'Product', 'Product_category', 'OFFER']]

In [11]:
# Create an empty dictionary to store brands and their associated sets of unique products
brand_product_dict = {}

# Create a dictionary to keep track of offers for each brand
brand_offers = {}

# Iterate through the rows in the dataframe
for _, row in ordered_df.iterrows():
    brand = row['BRAND']
    product = row['Product']
    offer = row['OFFER']

    # Check if an offer exists for the brand (ignore rows where offer is NaN)
    if not pd.isna(offer):
        # Check if the brand has an entry in the dictionary
        if brand not in brand_product_dict:
            brand_product_dict[brand] = set()  # Use a set to store unique products

        # Add the product to the brand's set of unique products
        brand_product_dict[brand].add(product)

    # Keep track of offers for each brand
    if brand not in brand_offers:
        brand_offers[brand] = []

    # Add the offer to the brand's list of offers
    brand_offers[brand].append(offer)

# Filter brands where all offers are NaN
brands_with_nan_offers = [brand for brand, offers in brand_offers.items() if all(pd.isna(offer) for offer in offers)]

# Now, brand_product_dict contains brands as keys and sets of unique products as values
# And brands_with_nan_offers contains brands with all NaN offers
print("Brand Product Dictionary:")
print(brand_product_dict)

print("\nBrands with All NaN Offers:")
print(brands_with_nan_offers)

Brand Product Dictionary:
{'caseys general store': {'jerky & dried meat', 'cookies', 'tea', 'tobacco products', 'chips', 'carbonated soft drinks', 'frozen pizza & pizza snacks', 'bakery', 'puffed snacks', 'beer', 'fruit juices', 'packaged vegetables', 'candy', 'cooking & baking', 'mature', 'household supplies', 'nuts & seeds', 'frozen desserts', 'energy drinks', 'water'}, 'dr pepper': {'carbonated soft drinks', 'cooking & baking'}, 'mountain dew': {'carbonated soft drinks'}, 'pepsi': {'carbonated soft drinks'}, 'cvs': {'skin care', 'cooking & baking', 'medicines & treatments', 'fruit juices'}, 'm&ms': {'candy'}, 'mcalisters deli': {'coffee', 'cookies', 'tea', 'dressings', 'sauces & marinades', 'chips', 'packaged meals & sides', 'wine', 'carbonated soft drinks', 'cheese', 'milk', 'packaged fruit & applesauce', 'bread', 'dips & salsa', 'bakery', 'frozen turkey', 'fruit juices', 'candy', 'pasta & noodles', 'leafy salads', 'frozen chicken', 'prepared meals', 'pickled goods', 'cooking & bak

In [14]:
def get_offers_for_brand(brand_name):
    # Filter the dataframe for the specified brand
    filtered_df = ordered_df[ordered_df['BRAND'] == brand_name]

    if filtered_df['OFFER'].isna().all():
        # Extract the products associated with the brand without offers
        products_for_brand_without_offers = set(filtered_df['Product'].unique())

        offer_similarities = []

        for other_brand, other_products in brand_product_dict.items():
            intersection = len(products_for_brand_without_offers.intersection(other_products))
            union = len(products_for_brand_without_offers.union(other_products))
            similarity = intersection / union

            if similarity > 0:
                offer_similarities.append((other_brand, similarity))

        if not offer_similarities:
            return "No similar brands with offers found."

        sorted_similar_brands = sorted(offer_similarities, key=lambda x: x[1], reverse=True)

        similar_brand_info = []
        for similar_brand, similarity_score in sorted_similar_brands:
            similar_brand_offers = brand_product_dict.get(similar_brand, [])
            similar_brand_actual_offers = ordered_df[ordered_df['BRAND'] == similar_brand]['OFFER'].unique()
            similar_brand_info.append({
                "Similar Brand": similar_brand,
                "Similarity Score": similarity_score,
                "Offers": similar_brand_actual_offers.tolist() if similar_brand_actual_offers.size > 0 else ["No offers"]
            })

        return similar_brand_info
    else:
        offers_for_brand = filtered_df['OFFER'].unique()
        unique_offers = list(set(offers_for_brand))

        if unique_offers:
            return [offer_line for offer_line in unique_offers]
        else:
            return "No offers found for the brand."

In [15]:
# Example usage:
brand_name = 'lays'  # Replace with the desired brand name
result = get_offers_for_brand(brand_name)
print(result)


[{'Similar Brand': 'tostitos', 'Similarity Score': 0.4, 'Offers': ['Tostitos® Toppers™']}, {'Similar Brand': 'nature valley', 'Similarity Score': 0.2727272727272727, 'Offers': ['Nature Valley™ Protein Granola']}, {'Similar Brand': 'bugles', 'Similarity Score': 0.25, 'Offers': ['General Mills™ snacks, select brands, spend $10']}, {'Similar Brand': 'natuchips', 'Similarity Score': 0.25, 'Offers': ['NatuChips® Plantain Chips']}, {'Similar Brand': 'emmys pop up', 'Similarity Score': 0.25, 'Offers': ["Emmy's Organics® Cookies OR Brownie Snaps"]}, {'Similar Brand': 'twix', 'Similarity Score': 0.2, 'Offers': ['TWIX®, select sizes', 'TWIX®, select varieties']}, {'Similar Brand': 'good humor', 'Similarity Score': 0.2, 'Offers': ['Good Humor Viennetta Frozen Vanilla Cake', "Reese's Peanut Butter Bar, 6 count, at GIANT OR MARTIN’S"]}, {'Similar Brand': 'brownie brittle', 'Similarity Score': 0.2, 'Offers': ['Brownie Brittle® Snacks, select varieties, buy 2']}, {'Similar Brand': 'hidden valley ranc